In [5]:
import random
import subprocess
import sys

# available samples
chr1_file = '1000Genomes/ALL.chr1.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz'
ls_samples = subprocess.check_output("bcftools query -l " + chr1_file, shell=True).decode(sys.stdout.encoding)
samples = ls_samples.split('\n')

# TODO: determine which are male/female (no X or Y chr files)
parents = random.sample(samples, 2)
print(parents)

['HG00379', 'HG00155']


In [19]:
def sim_recomb(parent_ids, num_offspring):
    for chrom in range(1, 23):
        chrom_file = '1000Genomes/ALL.chr' + str(chrom) + '.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz'
        queried_file = 'chr' + str(chrom) + '_parent_data.txt'
        # query_cmd = subprocess.check_output('bcftools query -f [\'%CHROM\t%POS\t%ID\t%REF\t%ALT\t%GT\n\'] -s ' + ','.join(parent_ids) + ' ' + chrom_file + ' > chr' + str(chrom) + '.txt', shell=True) #.decode(sys.stdout.encoding)
        # bcftools view -s HG00379,HG00155 1000Genomes/ALL.chr1.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz | awk '{print $1"\t"$2"\t"$3"\t"$4"\t"$5"\t"$10"\t"$11"\n"}'
        # bcftools query -f ['%CHROM\t%POS\t%ID\t%REF\t%ALT\t%GT\n'] -s HG00379,HG00155 1000Genomes/ALL.chr1.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz | awk  '{ if(NR%2 == 1) p = $6; else if (NR%2 == 0) print $0"\t"p; }' | awk 'BEGIN{print "CHROM\tPOS\tID\tREF\tALT\tPARENT1\tPARENT2"}1'
        bcf_cmd = 'bcftools query -f [\'%CHROM\t%POS\t%ID\t%REF\t%ALT\t%GT\n\'] -s ' + ','.join(parent_ids) + ' ' + chrom_file + ' | awk  \'{ if(NR%2 == 1) p = $6; else if (NR%2 == 0) print $0"\\t"p; }\' | awk \'BEGIN{print "CHROM\tPOS\tID\tREF\tALT\t' + parent_id[0] + '\\t' + parent_id[1] + '"}1\''
        min_pos = 'bcftools view -H ' + chrom_file + ' | awk \'(NR==1)'
        for o in range(num_offspring):
            # 0 - orig chromatid1, 1 - recomb chromatid1, 2 - orig chromatid2, 3 - recomb chromatid2
            chosen_chromatid = random.randint(0, 3)
            if chosen_chromatid % 2 == 0: # non-recombined

            else: # recombined
                # choose crossover point
                # find min and max pos values in file
                # randomly choose value in range
sim_recomb(parents, 1)

bcftools view -s HG00379,HG00155 1000Genomes/ALL.chr1.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz | awk '{print $1"\t"$2"\t"$3"\t"$4"\t"$5"\t"$10"\t"$11"\n"}' > chr1.txt


In [22]:
import pandas as pd
chr1_df = pd.read_csv('chr1.txt', sep='\t', names=['CHROM', 'POS', 'ID', 'REF', 'ALT', parents[0], parents[1]])

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [23]:
chr1_df

,CHROM,POS,ID,REF,ALT,HG00379,HG00155
0,##fileformat=VCFv4.1,NaN,NaN,NaN,NaN,NaN,NaN
1,"##FILTER=<ID=PASS,Description=""All",filters,"passed"">",NaN,NaN,NaN,NaN
2,##fileDate=20150218,NaN,NaN,NaN,NaN,NaN,NaN
3,##reference=ftp://ftp.1000genomes.ebi.ac.uk//v...,NaN,NaN,NaN,NaN,NaN,NaN
4,##source=1000GenomesPhase3Pipeline,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
6468344,1,249240217,rs570135855;rs556759920,G,"GT,T",0|0,0|0
6468345,1,249240219,rs370070114,A,T,1|0,0|0
6468346,1,249240537,rs549505823,GGT,G,0|0,0|0
6468347,1,249240539,rs545682552,T,G,0|1,0|1
